## Objectives

After completing this lab you will be able to:

-   Read CSV and JSON file types.
-   Extract data from the above file types.
-   Transform data.
-   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.

In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
def read_csv(file_to_process):
    dt_csv = pd.read_csv(file_to_process)
    return dt_csv

In [3]:
def read_json(file_to_process):
    dt_json = pd.read_json(file_to_process, lines = True)
    return dt_json

In [13]:
def read_xml(file_to_process):
    df = pd.DataFrame(columns=['name','height','weight'])
    
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    
    for person in root:
        name = person.find('name').text
        height = float(person.find('height').text)
        weight = float(person.find('weight').text)
        df = df.append({'name':name, 'height':height,'weight':weight}, ignore_index=True)
    return df

In [5]:
def extract():
    df_main = pd.DataFrame(columns=['name','height','weight'])
    
    #csv extract
    for fname in glob.glob('source_file_practice/*.csv'):
        df_main = df_main.append(read_csv(fname), ignore_index=True)
    
    #json extract
    for fname in glob.glob('source_file_practice/*.json'):
        df_main = df_main.append(read_json(fname), ignore_index=True)
    
    #xml extract
    for fname in glob.glob('source_file_practice/*.xml'):
        df_main = df_main.append(read_xml(fname), ignore_index=True)
    
    return df_main

In [6]:
def transform(data):
    data['height'] = data['height'].apply(lambda x : x * 0.0254)
    data['weight'] = data['weight'].apply(lambda x : x * 0.45359237)
    return data

In [7]:
def load(fname, data):
    data.to_csv('source_file_practice/' + fname)

In [8]:
def log(message):
    timestamp_format = '%y-%h-%d-%Y:%H-%D'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    
    with open('source_file_practice/logging.txt','a') as f:
        f.write(timestamp + ' ' + message + ' ' + '\n')

In [9]:
log('ETL Started')

In [10]:
log('Extract Begin')

In [14]:
df_extract = extract()

In [15]:
df_extract

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,jack,68.70,123.30
6,tom,69.80,141.49
7,tracy,70.01,136.46
8,john,67.90,112.37
9,simon,67.90,112.37


In [16]:
log('Extract End')

In [22]:
log('Transform Begin')

In [44]:
df_transform = transform(df_extract)

In [45]:
df_transform

,height,weight
name,,
alex,0.042439,23.247245
ajay,0.046142,28.082277
alice,0.044774,31.485316
ravi,0.044013,29.285891
joe,0.043735,29.689153
jack,0.044322,25.368487
tom,0.045032,29.111007
tracy,0.045168,28.076104
john,0.043806,23.119682


In [46]:
x = len(df_transform)

In [23]:
log('Transform End')

In [24]:
log('Load Begin')

In [25]:
load('BMI.csv', df_transform)

In [26]:
log('Load End')

In [27]:
log('ETL End')